In [46]:
import os
import pandas as pd
import time
import praw
from dotenv import load_dotenv
import re

load_dotenv()

True

In [47]:
reddit = praw.Reddit(
    client_id = os.getenv('client_id'),
    client_secret = os.getenv('client_secret'),
    user_agent = 'genalphaslang'
)

In [48]:
uoft_subreddit = reddit.subreddit('UofT')

Get top 100 posts comments

In [49]:
all_submissions = []
for submission in uoft_subreddit.top(limit = 200):
    submissions_dict = {}
    submissions_dict['date_created'] = submission.created_utc
    submissions_dict['title'] = submission.title
    submissions_dict['description'] = submission.selftext
    submission.comments.replace_more(limit = 0)
    submissions_dict['comments'] = [comment.body for comment in submission.comments.list()]
    submissions_dict['upvotes'] = submission.score

    # Add to all submissions
    all_submissions.append(submissions_dict)


In [50]:
df = pd.DataFrame(all_submissions)

In [51]:
df

,date_created,title,description,comments,upvotes
0,1.739385e+09,To the student who got caught using AI on thei...,Thanks a lot you dumb f*ck. Due to your idiocy...,"[[deleted], It’s crazy that there’s people mad...",2935
1,1.731036e+09,[ Removed by Reddit ],[ Removed by Reddit on account of violating th...,"[Ik this man, he’s getting cooked. Maybe uoft ...",2936
2,1.605297e+09,I want a ps5,,"[i LOVE how low effort this looks, pain, looo...",2752
3,1.481676e+09,If this thread gets over 300 upvotes in the ne...,"srsly don't fuck this up for me, we're already...","[[deleted], [deleted], Found this in /r/all/ri...",2485
4,1.744488e+09,uoft academic offence cases are literally so e...,,"[""The Panel does not accept this explanation a...",1991
...,...,...,...,...,...
195,1.567576e+09,ah shit,,[Dam i wish another winter like this would hap...,670
196,1.605824e+09,Uoft health insurance should cover back pain,,"[It needs to cover the pain of living too, I w...",673
197,1.590130e+09,My plans vs 2020,,"[Wow, PPPPM feels like ages ago., Remember whe...",664
198,1.612275e+09,The Boundary hilariously Roasting Rotman Students,,"[I'M WHEEZING, damn kid fked up., And another....",670


In [52]:
def combine_text(text): 
    if isinstance(text, list):
        return ' '.join([word for word in text if isinstance(word, str) and '[deleted]' not in word.lower()])
    elif isinstance(text, str): 
        return text
    return ''

In [53]:
df['combined_text'] = (df['title'].fillna('') + ' ' +
                       df['description'].fillna('') + ' ' +
                       df['comments'].apply(combine_text))

In [54]:
df['combined_text'] = df['combined_text'].str.lower().str.strip()

In [55]:
df['combined_text'][0]

'to the student who got caught using ai on their exam at u of t law thanks a lot you dumb f*ck. due to your idiocy and dumbassery my college (not in ontario) is now cracking down on access to our hard drives during exams, so now we have to print everything which will be a lot of time and money i don’t have. if you’re too incompetent to write a law school exam and require ai then you shouldn’t even be in law. thanks for ruining it for every other college across the country. it’s crazy that there’s people mad at you for venting. gaining access to your hard drives feels like a huge violation of privacy? i’m surprised that’s even allowed. ? [removed] what canadian says "college?" sighs, sounds exhausting. best of luck oof. i only use ai for helping me understand concepts (and actually backing up what it says with sources). idk why so many ppl use it to write their papers for them. it’s not that hard to write your own ideas! thanks to the idiot who got caught, there won’t be future dumb fuc

In [56]:
#Natural Language Processing Packages
import re
import nltk

## Download Resources
# nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download('averaged_perceptron_tagger_eng')
nltk.download("wordnet")
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gracelin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gracelin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/gracelin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gracelin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gracelin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/gracelin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [57]:
def normalize_uoft(text): #Need to add in other variations of uoft
    text = re.sub(r'u\sof\st', 'uoft', text)
    text = re.sub(r'university of toronto', 'uoft', text)
    return text

In [58]:
def tokenize_and_lemmatize(text): 
    tokens = word_tokenize(text)
    words = [word for word in tokens if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

In [59]:
df['combined_text'] = df['combined_text'].apply(normalize_uoft)
df['lemmatized_tokens'] = df['combined_text'].apply(tokenize_and_lemmatize)
df.head()

,date_created,title,description,comments,upvotes,combined_text,lemmatized_tokens
0,1.739385e+09,To the student who got caught using AI on thei...,Thanks a lot you dumb f*ck. Due to your idiocy...,"[[deleted], It’s crazy that there’s people mad...",2935,to the student who got caught using ai on thei...,"[student, got, caught, using, ai, exam, uoft, ..."
1,1.731036e+09,[ Removed by Reddit ],[ Removed by Reddit on account of violating th...,"[Ik this man, he’s getting cooked. Maybe uoft ...",2936,[ removed by reddit ] [ removed by reddit on a...,"[removed, reddit, removed, reddit, account, vi..."
2,1.605297e+09,I want a ps5,,"[i LOVE how low effort this looks, pain, looo...",2752,i want a ps5 i love how low effort this look...,"[want, love, low, effort, look, pain, loool, u..."
3,1.481676e+09,If this thread gets over 300 upvotes in the ne...,"srsly don't fuck this up for me, we're already...","[[deleted], [deleted], Found this in /r/all/ri...",2485,if this thread gets over 300 upvotes in the ne...,"[thread, get, upvotes, next, change, sub, exam..."
4,1.744488e+09,uoft academic offence cases are literally so e...,,"[""The Panel does not accept this explanation a...",1991,uoft academic offence cases are literally so e...,"[uoft, academic, offence, case, literally, ent..."


In [60]:
keywords = {"uoft", "university", "student", "professor", "campus", "course", "instructor", "lecture", "faculty", "ta"}

In [63]:
def is_uoft_related(tokens): 
    return bool(set(tokens) and keywords)

In [68]:
df_uoft = df[df['lemmatized_tokens'].apply(is_uoft_related)]
df_uoft['date_created'] = pd.to_datetime(df_uoft['date_created'], unit='s')
print(df_uoft.shape)
df_uoft.head()

(200, 7)


,date_created,title,description,comments,upvotes,combined_text,lemmatized_tokens
0,2025-02-12 18:36:14,To the student who got caught using AI on thei...,Thanks a lot you dumb f*ck. Due to your idiocy...,"[[deleted], It’s crazy that there’s people mad...",2935,to the student who got caught using ai on thei...,"[student, got, caught, using, ai, exam, uoft, ..."
1,2024-11-08 03:24:09,[ Removed by Reddit ],[ Removed by Reddit on account of violating th...,"[Ik this man, he’s getting cooked. Maybe uoft ...",2936,[ removed by reddit ] [ removed by reddit on a...,"[removed, reddit, removed, reddit, account, vi..."
2,2020-11-13 19:48:06,I want a ps5,,"[i LOVE how low effort this looks, pain, looo...",2752,i want a ps5 i love how low effort this look...,"[want, love, low, effort, look, pain, loool, u..."
3,2016-12-14 00:38:33,If this thread gets over 300 upvotes in the ne...,"srsly don't fuck this up for me, we're already...","[[deleted], [deleted], Found this in /r/all/ri...",2485,if this thread gets over 300 upvotes in the ne...,"[thread, get, upvotes, next, change, sub, exam..."
4,2025-04-12 20:03:29,uoft academic offence cases are literally so e...,,"[""The Panel does not accept this explanation a...",1991,uoft academic offence cases are literally so e...,"[uoft, academic, offence, case, literally, ent..."
